In [1]:
import IPython 
IPython.sys_info()

"{'commit_hash': '24103c4',\n 'commit_source': 'installation',\n 'default_encoding': 'utf-8',\n 'ipython_path': '/opt/miniconda3/envs/my_env/lib/python3.10/site-packages/IPython',\n 'ipython_version': '8.2.0',\n 'os_name': 'posix',\n 'platform': 'Linux-4.14.158-101.185.amzn1.x86_64-x86_64-with-glibc2.17',\n 'sys_executable': '/opt/miniconda3/envs/my_env/bin/python',\n 'sys_platform': 'linux',\n 'sys_version': '3.10.4 | packaged by conda-forge | (main, Mar 24 2022, '\n                '17:39:04) [GCC 10.3.0]'}"

In [2]:
import os
import pandas as pd
os.system("bamplot -h")

usage: bamplot [options]

optional arguments:
  -h, --help            show this help message and exit
  -b [BAM [BAM ...]], --bam [BAM [BAM ...]]
                        Enter a comma separated list of .bam files to be
                        processed.
  -i INPUT, --input INPUT
                        Enter .gff or genomic region e.g. chr1:+:1-1000.
  -g GENOME, --genome GENOME
                        specify a genome, HG18,HG19,MM8,MM9,MM10 are currently
                        supported
  -o OUTPUT, --output OUTPUT
                        Enter the output folder.
  --stretch-input STRETCH_INPUT
                        Stretch the input regions to a minimum length in bp,
                        e.g. 10000 (for 10kb)
  -c COLOR, --color COLOR
                        Enter a colon separated list of colors e.g.
                        255,0,0:255,125,0, default samples the rainbow
  -s SENSE, --sense SENSE
                        Map to '+','-' or 'both' strands. Default maps to
       

0

In [3]:
def bamplot_cmd(gene,region):
    data_table = pd.read_table("/data/projects/IRF4_mm_trn_data/myeloma_public_data/tables/p300_interal_perturb_data_table.txt")
    qc_table = pd.read_csv("/data/projects/IRF4_mm_trn_data/myeloma_public_data/tables/MM_p300_internal_QC_report.csv")

    order = data_table['UNIQUE_ID'].astype(str).str[:2]
    order = order.astype(int)
    data_table['order'] = order
    data_table = data_table.sort_values(by='order')

    order = qc_table['Sample'].astype(str).str[:2]
    order = order.astype(int)
    qc_table['order'] = order
    qc_table = qc_table.sort_values(by='order')

    selected_columns = qc_table[['Sample', 'order','Total_Reads_dm3']]

    data_table = data_table.merge(selected_columns, on='order', how='inner')

    data_table['time'] = [0,0,1,1,3,3,6,6,24,24,0,0,1,1,3,3,6,6,24,24,0,0,1,1,3,3,6,6,24,24,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]

    data_table['bam'] = data_table['FILE_PATH'] +data_table['UNIQUE_ID']+".bam"
    
    tf_bams = data_table['bam'][30:34].str.cat(sep=',')

    n0 = ["H3K18Ac_0","H3K18Ac_0","H3K27Ac_0","H3K27Ac_0","H3K27me3_0","H3K27me3_0"]

    cmd = "mkdir /home/ec2-user/IRF4_mm_trn/mm_internal_chip_scripts/figures/" + gene
    os.system(cmd)
    o = "/home/ec2-user/IRF4_mm_trn/mm_internal_chip_scripts/figures/"+ gene+"/"

    g = "hg19" 
    e = "2000" 
    c0 = "220,220,220:240,240,240"
    ct = "90,90,90:65,65,65:90,90,90:65,65,65:90,90,90:65,65,65:90,90,90:65,65,65"
    c = c0 +":"+ ct
    
    np300 = "IRF4,IRF4,p300,p300"
    
    cp300 = "244,64,247:31,200,222:51,232,232:45,208,208"
    y = "UNIFORM"
    p = "SINGLE"

    i = region
    title = "p300_IRF4_"+gene+"_bamplot"
   
    cmd = 'bamplot -g %s -e %s -c %s -o %s -y %s -b %s -i %s -n %s -t %s -r --save-temp -p %s \n\n' % (g,e,cp300,o,y,tf_bams,i,np300,title,p)
    #print(cmd)
    #os.system(cmd)
    
    #y = "RELATIVE"
    y = "UNIFORM"
    #p = "SINGLE"
    p = "MERGE"
    #-r 
    #--save-temp
    
    #### using spike-ins to normalize signal (scale input)
    
    for type_m in ["18Ac","27Ac","27me3"]:
        filtered_df = data_table[(data_table['NAME'].str.contains(type_m)) & (data_table.time == 0)]
        t0_bams = filtered_df['bam'].str.cat(sep=',')
        filtered_df['sf_mod'] = 1/(filtered_df['Total_Reads_dm3']/1000000)
        t0_sf = filtered_df['sf_mod'].astype('str').str.cat(sep=',')
        
        filtered_df = data_table[data_table.time != 0]
        filtered_df['sf_mod'] = 1/(filtered_df['Total_Reads_dm3']/1000000)
        filtered_df = filtered_df[filtered_df['NAME'].str.contains(type_m)]
        n_0 = pd.Series(n0)
        
        n_0 = n_0[n_0.str.contains(type_m)]
        n_t = "H3K" + type_m 
        
        n_0 = n_0.str.cat(sep=',')
        filtered_df['name'] = n_t 
        filtered_df['name'] = filtered_df['name'] +'_'+ filtered_df['time'].astype('str')
        n_t = filtered_df['name'].astype('str').str.cat(sep=',')
        n = n_0+","+ n_t
        t_bams = filtered_df['bam'].str.cat(sep=',')
        t_sf = filtered_df['sf_mod'].astype('str').str.cat(sep=',')

        b = t0_bams +','+ t_bams
        sf = t0_sf +','+ t_sf
        #print(sf)

        title = gene+"_"+type_m+"_bamplot_MM1S_scale"
        cmd = 'bamplot -g %s -e %s -c %s -o %s -y %s -b %s -i %s -n %s -t %s -r --save-temp -p %s --scale %s \n\n' % (g,e,c,o,y,b,i,n,title,p,sf)
        print(cmd)
        os.system(cmd)
        
        title = gene+"_"+type_m+"_bamplot_MM1S_no_scale"
        cmd = 'bamplot -g %s -e %s -c %s -o %s -y %s -b %s -i %s -n %s -t %s -r --save-temp -p %s \n\n' % (g,e,c,o,y,b,i,n,title,p)

        print(cmd)
        os.system(cmd)
        


In [7]:
region = "chr6:+:261715-458764"
gene ="IRF4"
bamplot_cmd(gene,region)


/tmp/ipykernel_12731/1957008385.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['sf_mod'] = 1/(filtered_df['Total_Reads_dm3']/1000000)
/tmp/ipykernel_12731/1957008385.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['sf_mod'] = 1/(filtered_df['Total_Reads_dm3']/1000000)
/tmp/ipykernel_12731/1957008385.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

bamplot -g hg19 -e 2000 -c 220,220,220:240,240,240:90,90,90:65,65,65:90,90,90:65,65,65:90,90,90:65,65,65:90,90,90:65,65,65 -o /home/ec2-user/IRF4_mm_trn/mm_internal_chip_scripts/figures/IRF4/ -y UNIFORM -b /data/projects/p300_internal_mm/01_0EWN_0229Kronos_MM1S-No-Treatment-1_H3K18Ac_hs-dm_i02.bam,/data/projects/p300_internal_mm/02_0EWO_0229Kronos_MM1S-No-Treatment-2_H3K18Ac_hs-dm_i04.bam,/data/projects/p300_internal_mm/03_0EWP_0229Kronos_MM1S-KB-1hr-1_H3K18Ac_hs-dm_i05.bam,/data/projects/p300_internal_mm/04_0EWQ_0229Kronos_MM1S-KB-1hr-2_H3K18Ac_hs-dm_i06.bam,/data/projects/p300_internal_mm/05_0EWR_0229Kronos_MM1S-KB-3hr-1_H3K18Ac_hs-dm_i07.bam,/data/projects/p300_internal_mm/06_0EWS_0229Kronos_MM1S-KB-3hr-2_H3K18Ac_hs-dm_i13.bam,/data/projects/p300_internal_mm/07_0EWT_0229Kronos_MM1S-KB-6hr-1_H3K18Ac_hs-dm_i14.bam,/data/projects/p300_internal_mm/08_0EWU_0229Kronos_MM1S-KB-6hr-2_H3K18Ac_hs-dm_i16.bam,/data/projects/p300_internal_mm/09_0EWV_0229Kronos_MM1S-KB-24hr-1_H3K18Ac_hs-dm_i17.ba

/tmp/ipykernel_12731/1957008385.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['sf_mod'] = 1/(filtered_df['Total_Reads_dm3']/1000000)
/tmp/ipykernel_12731/1957008385.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['sf_mod'] = 1/(filtered_df['Total_Reads_dm3']/1000000)
/tmp/ipykernel_12731/1957008385.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

bamplot -g hg19 -e 2000 -c 220,220,220:240,240,240:90,90,90:65,65,65:90,90,90:65,65,65:90,90,90:65,65,65:90,90,90:65,65,65 -o /home/ec2-user/IRF4_mm_trn/mm_internal_chip_scripts/figures/IRF4/ -y UNIFORM -b /data/projects/p300_internal_mm/11_0EWX_0229Kronos_MM1S-No-Treatment-1_H3K27Ac_hs-dm_i20.bam,/data/projects/p300_internal_mm/12_0EWY_0229Kronos_MM1S-No-Treatment-2_H3K27Ac_hs-dm_i21.bam,/data/projects/p300_internal_mm/13_0EWZ_0229Kronos_MM1S-KB-1hr-1_H3K27Ac_hs-dm_i22.bam,/data/projects/p300_internal_mm/14_0EX0_0229Kronos_MM1S-KB-1hr-2_H3K27Ac_hs-dm_i23.bam,/data/projects/p300_internal_mm/15_0EX1_0229Kronos_MM1S-KB-3hr-1_H3K27Ac_hs-dm_i25.bam,/data/projects/p300_internal_mm/16_0EX2_0229Kronos_MM1S-KB-3hr-2_H3K27Ac_hs-dm_i29.bam,/data/projects/p300_internal_mm/17_0EX3_0229Kronos_MM1S-KB-6hr-1_H3K27Ac_hs-dm_i33.bam,/data/projects/p300_internal_mm/18_0EX4_0229Kronos_MM1S-KB-6hr-2_H3K27Ac_hs-dm_i37.bam,/data/projects/p300_internal_mm/19_0EX5_0229Kronos_MM1S-KB-24hr-1_H3K27Ac_hs-dm_i39.ba

/tmp/ipykernel_12731/1957008385.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['sf_mod'] = 1/(filtered_df['Total_Reads_dm3']/1000000)
/tmp/ipykernel_12731/1957008385.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['sf_mod'] = 1/(filtered_df['Total_Reads_dm3']/1000000)


bamplot -g hg19 -e 2000 -c 220,220,220:240,240,240:90,90,90:65,65,65:90,90,90:65,65,65:90,90,90:65,65,65:90,90,90:65,65,65 -o /home/ec2-user/IRF4_mm_trn/mm_internal_chip_scripts/figures/IRF4/ -y UNIFORM -b /data/projects/p300_internal_mm/21_0F27_0229Kronos_MM1S-No-Treatment-1_H3K27me3_hs-dm_i02.bam,/data/projects/p300_internal_mm/22_0F28_0229Kronos_MM1S-No-Treatment-2_H3K27me3_hs-dm_i04.bam,/data/projects/p300_internal_mm/23_0F29_0229Kronos_MM1S-KB-1hr-1_H3K27me3_hs-dm_i05.bam,/data/projects/p300_internal_mm/24_0F2A_0229Kronos_MM1S-KB-1hr-2_H3K27me3_hs-dm_i06.bam,/data/projects/p300_internal_mm/25_0F2B_0229Kronos_MM1S-KB-3hr-1_H3K27me3_hs-dm_i07.bam,/data/projects/p300_internal_mm/26_0F2C_0229Kronos_MM1S-KB-3hr-2_H3K27me3_hs-dm_i13.bam,/data/projects/p300_internal_mm/27_0F2D_0229Kronos_MM1S-KB-6hr-1_H3K27me3_hs-dm_i14.bam,/data/projects/p300_internal_mm/28_0F2E_0229Kronos_MM1S-KB-6hr-2_H3K27me3_hs-dm_i16.bam,/data/projects/p300_internal_mm/29_0F2F_0229Kronos_MM1S-KB-24hr-1_H3K27me3_hs-

In [8]:
region = "chr5:+:87828193-88385787"
gene ="MEF2C"
bamplot_cmd(gene,region)

/tmp/ipykernel_12731/1957008385.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['sf_mod'] = 1/(filtered_df['Total_Reads_dm3']/1000000)
/tmp/ipykernel_12731/1957008385.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['sf_mod'] = 1/(filtered_df['Total_Reads_dm3']/1000000)
/tmp/ipykernel_12731/1957008385.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

bamplot -g hg19 -e 2000 -c 220,220,220:240,240,240:90,90,90:65,65,65:90,90,90:65,65,65:90,90,90:65,65,65:90,90,90:65,65,65 -o /home/ec2-user/IRF4_mm_trn/mm_internal_chip_scripts/figures/MEF2C/ -y UNIFORM -b /data/projects/p300_internal_mm/01_0EWN_0229Kronos_MM1S-No-Treatment-1_H3K18Ac_hs-dm_i02.bam,/data/projects/p300_internal_mm/02_0EWO_0229Kronos_MM1S-No-Treatment-2_H3K18Ac_hs-dm_i04.bam,/data/projects/p300_internal_mm/03_0EWP_0229Kronos_MM1S-KB-1hr-1_H3K18Ac_hs-dm_i05.bam,/data/projects/p300_internal_mm/04_0EWQ_0229Kronos_MM1S-KB-1hr-2_H3K18Ac_hs-dm_i06.bam,/data/projects/p300_internal_mm/05_0EWR_0229Kronos_MM1S-KB-3hr-1_H3K18Ac_hs-dm_i07.bam,/data/projects/p300_internal_mm/06_0EWS_0229Kronos_MM1S-KB-3hr-2_H3K18Ac_hs-dm_i13.bam,/data/projects/p300_internal_mm/07_0EWT_0229Kronos_MM1S-KB-6hr-1_H3K18Ac_hs-dm_i14.bam,/data/projects/p300_internal_mm/08_0EWU_0229Kronos_MM1S-KB-6hr-2_H3K18Ac_hs-dm_i16.bam,/data/projects/p300_internal_mm/09_0EWV_0229Kronos_MM1S-KB-24hr-1_H3K18Ac_hs-dm_i17.b

/tmp/ipykernel_12731/1957008385.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['sf_mod'] = 1/(filtered_df['Total_Reads_dm3']/1000000)
/tmp/ipykernel_12731/1957008385.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['sf_mod'] = 1/(filtered_df['Total_Reads_dm3']/1000000)
/tmp/ipykernel_12731/1957008385.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

bamplot -g hg19 -e 2000 -c 220,220,220:240,240,240:90,90,90:65,65,65:90,90,90:65,65,65:90,90,90:65,65,65:90,90,90:65,65,65 -o /home/ec2-user/IRF4_mm_trn/mm_internal_chip_scripts/figures/MEF2C/ -y UNIFORM -b /data/projects/p300_internal_mm/11_0EWX_0229Kronos_MM1S-No-Treatment-1_H3K27Ac_hs-dm_i20.bam,/data/projects/p300_internal_mm/12_0EWY_0229Kronos_MM1S-No-Treatment-2_H3K27Ac_hs-dm_i21.bam,/data/projects/p300_internal_mm/13_0EWZ_0229Kronos_MM1S-KB-1hr-1_H3K27Ac_hs-dm_i22.bam,/data/projects/p300_internal_mm/14_0EX0_0229Kronos_MM1S-KB-1hr-2_H3K27Ac_hs-dm_i23.bam,/data/projects/p300_internal_mm/15_0EX1_0229Kronos_MM1S-KB-3hr-1_H3K27Ac_hs-dm_i25.bam,/data/projects/p300_internal_mm/16_0EX2_0229Kronos_MM1S-KB-3hr-2_H3K27Ac_hs-dm_i29.bam,/data/projects/p300_internal_mm/17_0EX3_0229Kronos_MM1S-KB-6hr-1_H3K27Ac_hs-dm_i33.bam,/data/projects/p300_internal_mm/18_0EX4_0229Kronos_MM1S-KB-6hr-2_H3K27Ac_hs-dm_i37.bam,/data/projects/p300_internal_mm/19_0EX5_0229Kronos_MM1S-KB-24hr-1_H3K27Ac_hs-dm_i39.b

/tmp/ipykernel_12731/1957008385.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['sf_mod'] = 1/(filtered_df['Total_Reads_dm3']/1000000)
/tmp/ipykernel_12731/1957008385.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['sf_mod'] = 1/(filtered_df['Total_Reads_dm3']/1000000)
/tmp/ipykernel_12731/1957008385.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

bamplot -g hg19 -e 2000 -c 220,220,220:240,240,240:90,90,90:65,65,65:90,90,90:65,65,65:90,90,90:65,65,65:90,90,90:65,65,65 -o /home/ec2-user/IRF4_mm_trn/mm_internal_chip_scripts/figures/MEF2C/ -y UNIFORM -b /data/projects/p300_internal_mm/21_0F27_0229Kronos_MM1S-No-Treatment-1_H3K27me3_hs-dm_i02.bam,/data/projects/p300_internal_mm/22_0F28_0229Kronos_MM1S-No-Treatment-2_H3K27me3_hs-dm_i04.bam,/data/projects/p300_internal_mm/23_0F29_0229Kronos_MM1S-KB-1hr-1_H3K27me3_hs-dm_i05.bam,/data/projects/p300_internal_mm/24_0F2A_0229Kronos_MM1S-KB-1hr-2_H3K27me3_hs-dm_i06.bam,/data/projects/p300_internal_mm/25_0F2B_0229Kronos_MM1S-KB-3hr-1_H3K27me3_hs-dm_i07.bam,/data/projects/p300_internal_mm/26_0F2C_0229Kronos_MM1S-KB-3hr-2_H3K27me3_hs-dm_i13.bam,/data/projects/p300_internal_mm/27_0F2D_0229Kronos_MM1S-KB-6hr-1_H3K27me3_hs-dm_i14.bam,/data/projects/p300_internal_mm/28_0F2E_0229Kronos_MM1S-KB-6hr-2_H3K27me3_hs-dm_i16.bam,/data/projects/p300_internal_mm/29_0F2F_0229Kronos_MM1S-KB-24hr-1_H3K27me3_hs

In [9]:
region = "chr17:+:37847100-38060047"
gene ="IKZF3"
bamplot_cmd(gene,region)

/tmp/ipykernel_12731/1957008385.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['sf_mod'] = 1/(filtered_df['Total_Reads_dm3']/1000000)
/tmp/ipykernel_12731/1957008385.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['sf_mod'] = 1/(filtered_df['Total_Reads_dm3']/1000000)
/tmp/ipykernel_12731/1957008385.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

bamplot -g hg19 -e 2000 -c 220,220,220:240,240,240:90,90,90:65,65,65:90,90,90:65,65,65:90,90,90:65,65,65:90,90,90:65,65,65 -o /home/ec2-user/IRF4_mm_trn/mm_internal_chip_scripts/figures/IKZF3/ -y UNIFORM -b /data/projects/p300_internal_mm/01_0EWN_0229Kronos_MM1S-No-Treatment-1_H3K18Ac_hs-dm_i02.bam,/data/projects/p300_internal_mm/02_0EWO_0229Kronos_MM1S-No-Treatment-2_H3K18Ac_hs-dm_i04.bam,/data/projects/p300_internal_mm/03_0EWP_0229Kronos_MM1S-KB-1hr-1_H3K18Ac_hs-dm_i05.bam,/data/projects/p300_internal_mm/04_0EWQ_0229Kronos_MM1S-KB-1hr-2_H3K18Ac_hs-dm_i06.bam,/data/projects/p300_internal_mm/05_0EWR_0229Kronos_MM1S-KB-3hr-1_H3K18Ac_hs-dm_i07.bam,/data/projects/p300_internal_mm/06_0EWS_0229Kronos_MM1S-KB-3hr-2_H3K18Ac_hs-dm_i13.bam,/data/projects/p300_internal_mm/07_0EWT_0229Kronos_MM1S-KB-6hr-1_H3K18Ac_hs-dm_i14.bam,/data/projects/p300_internal_mm/08_0EWU_0229Kronos_MM1S-KB-6hr-2_H3K18Ac_hs-dm_i16.bam,/data/projects/p300_internal_mm/09_0EWV_0229Kronos_MM1S-KB-24hr-1_H3K18Ac_hs-dm_i17.b

/tmp/ipykernel_12731/1957008385.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['sf_mod'] = 1/(filtered_df['Total_Reads_dm3']/1000000)
/tmp/ipykernel_12731/1957008385.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['sf_mod'] = 1/(filtered_df['Total_Reads_dm3']/1000000)
/tmp/ipykernel_12731/1957008385.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

bamplot -g hg19 -e 2000 -c 220,220,220:240,240,240:90,90,90:65,65,65:90,90,90:65,65,65:90,90,90:65,65,65:90,90,90:65,65,65 -o /home/ec2-user/IRF4_mm_trn/mm_internal_chip_scripts/figures/IKZF3/ -y UNIFORM -b /data/projects/p300_internal_mm/11_0EWX_0229Kronos_MM1S-No-Treatment-1_H3K27Ac_hs-dm_i20.bam,/data/projects/p300_internal_mm/12_0EWY_0229Kronos_MM1S-No-Treatment-2_H3K27Ac_hs-dm_i21.bam,/data/projects/p300_internal_mm/13_0EWZ_0229Kronos_MM1S-KB-1hr-1_H3K27Ac_hs-dm_i22.bam,/data/projects/p300_internal_mm/14_0EX0_0229Kronos_MM1S-KB-1hr-2_H3K27Ac_hs-dm_i23.bam,/data/projects/p300_internal_mm/15_0EX1_0229Kronos_MM1S-KB-3hr-1_H3K27Ac_hs-dm_i25.bam,/data/projects/p300_internal_mm/16_0EX2_0229Kronos_MM1S-KB-3hr-2_H3K27Ac_hs-dm_i29.bam,/data/projects/p300_internal_mm/17_0EX3_0229Kronos_MM1S-KB-6hr-1_H3K27Ac_hs-dm_i33.bam,/data/projects/p300_internal_mm/18_0EX4_0229Kronos_MM1S-KB-6hr-2_H3K27Ac_hs-dm_i37.bam,/data/projects/p300_internal_mm/19_0EX5_0229Kronos_MM1S-KB-24hr-1_H3K27Ac_hs-dm_i39.b

/tmp/ipykernel_12731/1957008385.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['sf_mod'] = 1/(filtered_df['Total_Reads_dm3']/1000000)
/tmp/ipykernel_12731/1957008385.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['sf_mod'] = 1/(filtered_df['Total_Reads_dm3']/1000000)
/tmp/ipykernel_12731/1957008385.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

bamplot -g hg19 -e 2000 -c 220,220,220:240,240,240:90,90,90:65,65,65:90,90,90:65,65,65:90,90,90:65,65,65:90,90,90:65,65,65 -o /home/ec2-user/IRF4_mm_trn/mm_internal_chip_scripts/figures/IKZF3/ -y UNIFORM -b /data/projects/p300_internal_mm/21_0F27_0229Kronos_MM1S-No-Treatment-1_H3K27me3_hs-dm_i02.bam,/data/projects/p300_internal_mm/22_0F28_0229Kronos_MM1S-No-Treatment-2_H3K27me3_hs-dm_i04.bam,/data/projects/p300_internal_mm/23_0F29_0229Kronos_MM1S-KB-1hr-1_H3K27me3_hs-dm_i05.bam,/data/projects/p300_internal_mm/24_0F2A_0229Kronos_MM1S-KB-1hr-2_H3K27me3_hs-dm_i06.bam,/data/projects/p300_internal_mm/25_0F2B_0229Kronos_MM1S-KB-3hr-1_H3K27me3_hs-dm_i07.bam,/data/projects/p300_internal_mm/26_0F2C_0229Kronos_MM1S-KB-3hr-2_H3K27me3_hs-dm_i13.bam,/data/projects/p300_internal_mm/27_0F2D_0229Kronos_MM1S-KB-6hr-1_H3K27me3_hs-dm_i14.bam,/data/projects/p300_internal_mm/28_0F2E_0229Kronos_MM1S-KB-6hr-2_H3K27me3_hs-dm_i16.bam,/data/projects/p300_internal_mm/29_0F2F_0229Kronos_MM1S-KB-24hr-1_H3K27me3_hs